In [23]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Input

from keras.callbacks import History 

from keras import optimizers
from keras.datasets import imdb

In questo notebook addestreremo una rete neurale in grado di comprendere se una recensione relativa a un film è positiva o negativa. 
A questo scopo utilizzeremo l'IMDB Moview Review Dataset un dataset di 50.000 recensioni etichettate come positive o negative.
Possiamo caricare il dataset utilizzando direttamente Keras, il parametro num_words ci serve per impostare il numero massimo di parole più frequenti da selezionare e di conseguenza il numero di features del nostro modello.

In [5]:
(X_train, y_train) , (X_test, y_test) = imdb.load_data(num_words=5000)  #numero massimo di parole più frequenti

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
print(X_train.shape)
print(X_test.shape)
X_train[0]      #contiene il numero delle parole più frequenti, alla posizione zero abbiamo la prima parola più frequente

(25000,)
(25000,)


[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 2,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 2,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 2,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 2,
 18,
 51,
 36,
 

In [13]:
word_index = imdb.get_word_index()      #creo un dizionario che mi restituisce il numero di volte che compare una parola
print(word_index['love']-3)

113


In [16]:
#per risalire dal numero alla parola devo invertire l'ultimo dizionario
reverse_word_index = dict((value, key) for (key, value) in (word_index.items()))    #inverto i valori con le chiavi
print(reverse_word_index.get(113+3))

love


In [20]:
#a causa di alcuni spazi e caratteri ogni posizione è shiftata di 3 posizioni
decoded_review = [reverse_word_index.get(i-3,'?') for i in X_train[0]]      #dizionario di parole
print(decoded_review)
print(' '.join(decoded_review))         #frase intera

['?', 'this', 'film', 'was', 'just', 'brilliant', 'casting', 'location', 'scenery', 'story', 'direction', "everyone's", 'really', 'suited', 'the', 'part', 'they', 'played', 'and', 'you', 'could', 'just', 'imagine', 'being', 'there', 'robert', '?', 'is', 'an', 'amazing', 'actor', 'and', 'now', 'the', 'same', 'being', 'director', '?', 'father', 'came', 'from', 'the', 'same', 'scottish', 'island', 'as', 'myself', 'so', 'i', 'loved', 'the', 'fact', 'there', 'was', 'a', 'real', 'connection', 'with', 'this', 'film', 'the', 'witty', 'remarks', 'throughout', 'the', 'film', 'were', 'great', 'it', 'was', 'just', 'brilliant', 'so', 'much', 'that', 'i', 'bought', 'the', 'film', 'as', 'soon', 'as', 'it', 'was', 'released', 'for', '?', 'and', 'would', 'recommend', 'it', 'to', 'everyone', 'to', 'watch', 'and', 'the', 'fly', '?', 'was', 'amazing', 'really', 'cried', 'at', 'the', 'end', 'it', 'was', 'so', 'sad', 'and', 'you', 'know', 'what', 'they', 'say', 'if', 'you', 'cry', 'at', 'a', 'film', 'it', '

### Preprocessing

Le frasi nel corpus di testo sono già codificate in numeri, ma hanno una lunghezza differente, per poterle utilizzare per addestrare la nostra rete neurale abbiamo bisogno di codificarle per ottenere un numero di features consistente. Abbiamo diverse tecniche per farlo, utilizziamo la più semplice: il one hot encoding.

In [21]:
def one_hot_encoding(data, size):
    onehot = np.zeros((len(data), size))    #creo una matrice di zeri dato la dimensione del dataset (riga) e il numero del dataset (colonna)
    for i, d in enumerate(data):            #itero sugli esempi e con enumerate posso ricavare sia il valore che l'indice
        onehot[i,d] = 1.
    return onehot

In [22]:
X_train_encoded = one_hot_encoding(X_train, 5000)       #utilizzo la funzione sul set di train con numero di parole = 5000
X_test_encoded = one_hot_encoding(X_test, 5000)

print(X_train_encoded.shape)

(25000, 5000)


### Creiamo il modello

In [27]:
model = Sequential()
model.add(Input(shape=(X_train_encoded.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

E' un problema di classificazione binaria (recensione positiva o negativa)

In [28]:
#utilizziamo come ottimizzatore l'adamax anzichè l'adam
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_encoded, y_train, epochs=10, batch_size=512)

Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.7625 - loss: 0.5067
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9184 - loss: 0.2172
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9419 - loss: 0.1648
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9614 - loss: 0.1216
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9808 - loss: 0.0764
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9940 - loss: 0.0404
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9975 - loss: 0.0217
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9992 - loss: 0.0096
Epoch 9/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9996 - loss: 0.0057
Epoch 10/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9997 - loss: 0.0038


In [29]:
#risultati ottimi, provo sul test per vedere se si tratta di overfitting
model.evaluate(X_test_encoded, y_test)          #OVERFITTING: loss di test >> loss di train

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8648 - loss: 0.5912


[0.5740670561790466, 0.8696799874305725]

### Combattiamo l'overfitting con la regolarizzazione e il dropout

In [30]:
from keras.regularizers import l2
from keras.layers import Dropout

model = Sequential()
model.add(Input(shape=(X_train_encoded.shape[1],)))
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.1)))    #per il primo strato più intensa e meno intensa per gli altri
model.add(Dropout(0.5))    #0 non rimuove nessun neurone, 1 li rimuove tutti
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5)) 
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5)) 
model.add(Dense(8, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5)) 
model.add(Dense(1, activation='sigmoid'))

In [31]:
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_encoded, y_train, epochs=100, batch_size=512)      #aumento il numero di epoche a 100

Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5052 - loss: 45.4247
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5874 - loss: 1.4941
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6598 - loss: 0.9453
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7270 - loss: 0.7827
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.7653 - loss: 0.7155
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7853 - loss: 0.6885
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.7972 - loss: 0.6627
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8159 - loss: 0.6305
Epoch 9/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8264 - loss: 0.6211
Epoch 10/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.8399 - loss: 0.6076
Epoch 11/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8440 - loss: 0.5921
Epoch 12/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/ste

In [32]:
model.evaluate(X_test_encoded, y_test)      #problema risolto diciamo -> potevamo risolvere meglio con algoritmi per NLP (RNN o bag of words)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8698 - loss: 0.5062


[0.5051153302192688, 0.8705199956893921]

### Proviamo il nostro modello

In [33]:
#prendo una recensione a caso di un film brutto
review = "The movie is boring and predictable, with a weak script and underdeveloped characters. The acting feels forced, and the slow pace makes it hard to watch until the end. A disappointing experience"
#dobbiamo ottenere dalle parole gli indici
from re import sub
review = sub(r'[^\w\s]','',review)      #rimuovo tutta la punteggiatura non necessaria al nostro scopo
review = review.lower()                 #converto tutto in lower case
review = review.split(" ")              #converto la stringa in un array di parole
print(review)

['the', 'movie', 'is', 'boring', 'and', 'predictable', 'with', 'a', 'weak', 'script', 'and', 'underdeveloped', 'characters', 'the', 'acting', 'feels', 'forced', 'and', 'the', 'slow', 'pace', 'makes', 'it', 'hard', 'to', 'watch', 'until', 'the', 'end', 'a', 'disappointing', 'experience']


Quale di queste parole rientra nelle 5000 parole più frequenti nel nostro corpus di testo?

In [35]:
review_array = []       #serve per collezionare gli indici delle parole più frequenti
for word in review:
    if word in word_index:
        index = word_index[word]
        if index <= 5000:
            review_array.append(index + 3)      #eseguo lo shift di 3 per lo stesso discorso dell'inizio
review_array

[4,
 20,
 9,
 357,
 5,
 727,
 19,
 6,
 815,
 229,
 5,
 105,
 4,
 116,
 764,
 918,
 5,
 4,
 550,
 1062,
 166,
 12,
 254,
 8,
 106,
 366,
 4,
 130,
 6,
 1332,
 585]

In [37]:
#eseguo il onehot-encoding
x = one_hot_encoding([review_array], 5000)
x.shape

(1, 5000)

In [39]:
prob = model.predict(x)
prob            #probabilità del 2% che sia positiva e del 98% che sia negativa

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[0.02766763]], dtype=float32)

In [40]:
#scrivo una funzione che stampa il sentiment a partire dalla probabilità
def prob_to_sentiment(y):
    
    if(prob>0.9): return "fantastica"
    elif(prob>0.75): return "ottima"
    elif(prob>0.55): return "buona" 
    elif(prob>0.45): return "neutrale"
    elif(prob>0.25): return "negativa"
    elif(prob>0.1): return "brutta"
    else: return "pessima"

In [41]:
prob_to_sentiment(prob)

'pessima'

Proviamo un'altra recensione

In [42]:
review = "This movie will blow your mind and break your heart - and make you desperate to go back for more. Brave, brilliant and better than it has any right to be."
review = sub(r'[^\w\s]','',review)      #rimuovo tutta la punteggiatura non necessaria al nostro scopo
review = review.lower()                 #converto tutto in lower case
review = review.split(" ") 

review_array = []       #serve per collezionare gli indici delle parole più frequenti
for word in review:
    if word in word_index:
        index = word_index[word]
        if index <= 5000:
            review_array.append(index + 3)      #eseguo lo shift di 3 per lo stesso discorso dell'inizio
x = one_hot_encoding([review_array], 5000)
prob = model.predict(x)
prob_to_sentiment(prob)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


'fantastica'

### Test di validazione

Testare sempre il modello sul dataset di test può portare ad overfitting su quest'ultimo poichè il modello può iniziare a considerarlo come una parte del training set. Quindi è utile prendere una parte del dataset solamente per la validazione e per perfezionare gli iperparametri della rete